In [ ]:
#Importing the necessary classes
import pandas as pd
import numpy as np
import numpy as np
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
from matplotlib import rcParams
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
#Loading the time series dataset
dataset= pd.read_csv('/content/2608688.csv')
dataset[["DATE"]] = dataset[["DATE"]].apply(pd.to_datetime)

In [ ]:
# Shifting the precipitation at different forecasting horizons: An example of 4-day prior forecast horizon
Rain4= dataset.iloc[:,2:12]
Rain_one4= dataset.iloc[:,2:12]
Rain4.drop(Rain4.tail(4).index,inplace=True)
Rain_one4.drop(Rain_one4.head(4).index,inplace=True)
Rain4.reset_index(drop=True, inplace=True)
Rain_one4.reset_index(drop=True, inplace=True)

In [ ]:
Rain4[0:5]

,DATE,AWND,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5
0,2015-01-01,12.30,0,21,28,17,260,260.0,21.9,33.1
1,2015-02-01,4.03,0,19,25,8,240,260.0,15.0,19.9
2,2015-03-01,5.82,1,21,32,10,130,140.0,14.1,19.0
3,2015-04-01,10.07,1,28,32,8,320,310.0,21.9,30.0
4,2015-05-01,9.62,1,7,9,1,270,270.0,23.0,31.1


In [ ]:
Rain_one4.head(5)

,DATE,AWND,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5
0,2015-05-01,9.62,1,7,9,1,270,270.0,23.0,31.1
1,2015-06-01,10.29,0,7,16,2,240,230.0,18.1,25.9
2,2015-07-01,13.65,1,5,10,-2,320,330.0,28.0,40.0
3,2015-08-01,13.65,1,9,16,5,180,260.0,21.0,27.1
4,2015-09-01,9.40,1,8,12,2,260,300.0,18.1,25.1


In [ ]:
Rain_oneday4= Rain4[['TAVG','TMAX', 'TMIN','AWND', 'WDF2', 'WDF5', 'WSF2','WSF5','PRCP']]
Rain_oneday4['Final']= Rain_one4['PRCP']

In [ ]:
Rain_oneday4.head(10)

,TAVG,TMAX,TMIN,AWND,WDF2,WDF5,WSF2,WSF5,PRCP,Final
0,21,28,17,12.30,260,260.0,21.9,33.1,0,1
1,19,25,8,4.03,240,260.0,15.0,19.9,0,0
2,21,32,10,5.82,130,140.0,14.1,19.0,1,1
3,28,32,8,10.07,320,310.0,21.9,30.0,1,1
4,7,9,1,9.62,270,270.0,23.0,31.1,1,1
5,7,16,2,10.29,240,230.0,18.1,25.9,0,0
6,5,10,-2,13.65,320,330.0,28.0,40.0,1,0
7,9,16,5,13.65,180,260.0,21.0,27.1,1,1
8,8,12,2,9.40,260,300.0,18.1,25.1,1,0
9,6,18,0,9.62,250,240.0,21.0,30.0,0,0


In [ ]:
# Code for removing the null observations
for index, row in Rain_oneday4.iterrows():
  a = row[6]
  if math.isnan(a):
    Rain_oneday4.drop([index-1], axis=0, inplace=True)
    Rain_oneday4.drop([index], axis=0, inplace=True)
    continue

In [ ]:
#Train and test datasets
features= Rain_oneday4.iloc[:, 1:7].values
labels= Rain_oneday4.iloc[:,-1].values
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.15,  shuffle=False)

In [ ]:
#Training the SVM model and getting the RMSE values
svm= SVC()
svm.fit(x_train, y_train)
y_pred_svr= svm.predict(x_test)
print(np.sqrt(mean_squared_error(y_test,y_pred_svr)))
accuracy_score(y_test,y_pred_svr)

In [ ]:
#Important feature identification
dataset['PRCP'][dataset['PRCP'] > 0] = 1
dataset['PRCP'] = dataset['PRCP'].apply(np.int64)
dataset.dropna(inplace=True)
model=LogisticRegression(random_state=1)
model.fit(X,y)
feature_importance=pd.DataFrame({'feature':list(X.columns),'feature_importance':[abs(i) for i in model.coef_[0]]})
feature_importance.sort_values('feature_importance',ascending=False)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,feature,feature_importance
3,AWND,0.119682
2,TMIN,0.111796
1,TMAX,0.100219
6,WSF2,0.075578
7,WSF5,0.057313
0,TAVG,0.008760
4,WDF2,0.001055
5,WDF5,0.000429
